In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#%load_ext autoreload
#%autoreload 2

import os
#change to desired path
os.chdir("/content/drive/MyDrive/capstone")
%pwd

In [ ]:
import os
os.environ['HF_HOME'] = '/content/drive/MyDrive/capstone/huggingface'

In [ ]:
!echo $HF_HOME

In [ ]:
!pip install eai-sparsify

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer

from sparsify import SaeConfig, Trainer, TrainConfig
from sparsify.data import chunk_and_tokenize
from tqdm import tqdm
from multiprocessing import cpu_count

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon MPS device.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using NVIDIA CUDA device.")
else:
    device = torch.device("cpu")
    print("Using CPU device.")

In [ ]:
dataset = load_dataset(
    "EleutherAI/SmolLM2-135M-10B",
    split="train",
)

In [ ]:
model_name = "Qwen/Qwen3-Embedding-0.6B"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    #device=device,
    #device_map={"": str(device)},
    device=torch.device("cpu"), # Data was tokenized on a v6e-1 TPU runtime loaded on cpu, need to keep to load from cache
    device_map={"": "cpu"}, # Data was tokenized on a v6e-1 TPU runtime loaded on cpu, need to keep to load from cache
)

In [ ]:
tokenized = chunk_and_tokenize(
    dataset,
    tokenizer,
    #num_proc = cpu_count(),
    num_proc = 44 # Data was tokenized on a v6e-1 TPU runtime with 44 processes, need to keep to load from cache
)

In [ ]:
model = AutoModel.from_pretrained(
    model_name,
    # device=device,
    device_map={"": str(device)},
)

In [ ]:
sae_config = SaeConfig()
sae_config.expansion_factor = 16
sae_config.transcode = False

In [ ]:
train_config = TrainConfig(sae_config)
train_config.log_to_wandb = False
train_config.run_name = model_name.replace('/', '-')
train_config.hookpoints = ["layers.*.mlp"]
train_config.batch_size = 64
train_config.micro_acc_steps = 2

In [ ]:
trainer = Trainer(train_config, tokenized, model)

In [ ]:
trainer.fit()